In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import pandas as pd
from IPython.display import display
pd.options.display.max_columns = None
from sklearn.preprocessing import Imputer
from catboost import CatBoostRegressor, Pool, cv

In [2]:
X_train = pd.read_csv('dengue_features_train.csv', index_col=None)
y_train = pd.read_csv('dengue_labels_train.csv', index_col=None)
X_test = pd.read_csv('dengue_features_test.csv', index_col=None)

In [3]:
X_train['week_start_date'] = pd.to_datetime(X_train['week_start_date'])
X_test['week_start_date'] = pd.to_datetime(X_test['week_start_date'])

In [4]:
X_train['month'] = X_train['week_start_date'].dt.month
X_train['quarter'] = X_train['week_start_date'].dt.quarter
X_train = X_train.drop(['week_start_date'], axis=1)

In [5]:
X_test['month'] = X_test['week_start_date'].dt.month
X_test['quarter'] = X_test['week_start_date'].dt.quarter
X_test = X_test.drop(['week_start_date'], axis=1)

In [14]:
labels = X_train['city'].unique().tolist()
mapping = dict(zip(labels,range(len(labels))))
X_train.replace({'city': mapping},inplace=True)

In [26]:
labels = X_test['city'].unique().tolist()
mapping = dict(zip(labels,range(len(labels))))
X_test.replace({'city': mapping},inplace=True)

In [23]:
y_train = y_train['total_cases']

In [11]:
cat_features = [0,1,2,23,24]

In [20]:
X_train.head()

,city,year,weekofyear,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_min_air_temp_k,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,month,quarter
0,0,1990,18,0.122600,0.103725,0.198483,0.177617,12.42,297.572857,297.742857,292.414286,299.8,295.9,32.00,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0,4,2
1,0,1990,19,0.169900,0.142175,0.162357,0.155486,22.82,298.211429,298.442857,293.951429,300.9,296.4,17.94,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6,5,2
2,0,1990,20,0.032250,0.172967,0.157200,0.170843,34.54,298.781429,298.878571,295.434286,300.5,297.3,26.10,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4,5,2
3,0,1990,21,0.128633,0.245067,0.227557,0.235886,15.36,298.987143,299.228571,295.310000,301.4,297.0,13.90,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0,5,2
4,0,1990,22,0.196200,0.262200,0.251200,0.247340,7.52,299.518571,299.664286,295.821429,301.9,297.5,12.20,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8,5,2


In [10]:
model = CatBoostRegressor(iterations=3, learning_rate=0.01, depth=3)

In [25]:
model.fit(X_train, y_train, cat_features)

In [27]:
result = model.predict(X_test)

In [28]:
X_test = pd.read_csv('dengue_features_test.csv', index_col=None)
result_columns = X_test[['city','year','weekofyear']]
result_cases = pd.DataFrame(result)
result_cases.columns = ['total_cases']
result_df = pd.concat([result_columns, result_cases], axis=1)
result_df['total_cases'] = result_df['total_cases'].round().astype(int)

In [29]:
result_df['total_cases'] = np.where(result_df['total_cases']<0, 0, result_df['total_cases'])

In [30]:
result_df.head()

,city,year,weekofyear,total_cases
0,sj,2008,18,0
1,sj,2008,19,0
2,sj,2008,20,0
3,sj,2008,21,0
4,sj,2008,22,0


In [31]:
result_df.to_csv('catboost.csv', index=False)